In [ ]:
!python -m pip install --upgrade pip
!python -m pip install pip==20.2.4

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-geometric

In [ ]:
import numpy as np
import networkx as nx
import os
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Embedding
from torch.nn import Parameter
from torch_geometric.data import Data,DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.utils.convert import to_networkx
from torch_geometric.utils import to_undirected

In [ ]:
os.environ['KAGGLE_USERNAME'] = "karthikapv" # username from the json file
os.environ['KAGGLE_KEY'] = "cc11b8fcbb2e177d31cd566bbabe382a" # key from the json file
!kaggle datasets download -d ellipticco/elliptic-data-set
!unzip elliptic-data-set.zip
!mkdir elliptic_bitcoin_dataset_cont

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import to_undirected


def train_test_split():
    df_edge = pd.read_csv('elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv')
    df_class = pd.read_csv('elliptic_bitcoin_dataset/elliptic_txs_classes.csv')
    df_features = pd.read_csv('elliptic_bitcoin_dataset/elliptic_txs_features.csv', header=None)

    # Setting Column name
    df_features.columns = ['id', 'time step'] + [f'trans_feat_{i}' for i in range(93)] + [f'agg_feat_{i}' for i in
                                                                                          range(72)]

    print('Number of edges: {}'.format(len(df_edge)))
    df_edge.head()

    # Get Node Index

    all_nodes = list(
        set(df_edge['txId1']).union(set(df_edge['txId2'])).union(set(df_class['txId'])).union(set(df_features['id'])))
    nodes_df = pd.DataFrame(all_nodes, columns=['id']).reset_index()

    print('Number of nodes: {}'.format(len(nodes_df)))
    nodes_df.head()

    # Fix id index

    df_edge = df_edge.join(nodes_df.rename(columns={'id': 'txId1'}).set_index('txId1'), on='txId1', how='inner') \
        .join(nodes_df.rename(columns={'id': 'txId2'}).set_index('txId2'), on='txId2', how='inner', rsuffix='2') \
        .drop(columns=['txId1', 'txId2']) \
        .rename(columns={'index': 'txId1', 'index2': 'txId2'})
    df_edge.head()

    df_class = df_class.join(nodes_df.rename(columns={'id': 'txId'}).set_index('txId'), on='txId', how='inner') \
        .drop(columns=['txId']).rename(columns={'index': 'txId'})[['txId', 'class']]
    df_class.head()

    df_features = df_features.join(nodes_df.set_index('id'), on='id', how='inner') \
        .drop(columns=['id']).rename(columns={'index': 'id'})
    df_features = df_features[['id'] + list(df_features.drop(columns=['id']).columns)]
    df_features.head()

    df_edge_time = df_edge.join(df_features[['id', 'time step']].rename(columns={'id': 'txId1'}).set_index('txId1'),
                                on='txId1', how='left', rsuffix='1') \
        .join(df_features[['id', 'time step']].rename(columns={'id': 'txId2'}).set_index('txId2'), on='txId2', how='left',
              rsuffix='2')
    df_edge_time['is_time_same'] = df_edge_time['time step'] == df_edge_time['time step2']
    df_edge_time_fin = df_edge_time[['txId1', 'txId2', 'time step']].rename(
        columns={'txId1': 'source', 'txId2': 'target', 'time step': 'time'})

    # Create csv from Dataframe

    df_features.drop(columns=['time step']).to_csv('elliptic_bitcoin_dataset_cont/elliptic_txs_features.csv', index=False, header=None)
    df_class.rename(columns={'txId': 'nid', 'class': 'label'})[['nid', 'label']].sort_values(by='nid').to_csv(
        'elliptic_bitcoin_dataset_cont/elliptic_txs_classes.csv', index=False, header=None)
    df_features[['id', 'time step']].rename(columns={'id': 'nid', 'time step': 'time'})[['nid', 'time']].sort_values(
        by='nid').to_csv('elliptic_bitcoin_dataset_cont/elliptic_txs_nodetime.csv', index=False, header=None)
    df_edge_time_fin[['source', 'target', 'time']].to_csv('elliptic_bitcoin_dataset_cont/elliptic_txs_edgelist_timed.csv', index=False,
                                                          header=None)

    # Graph Preprocessing

    node_label = df_class.rename(columns={'txId': 'nid', 'class': 'label'})[['nid', 'label']].sort_values(by='nid').merge(
        df_features[['id', 'time step']].rename(columns={'id': 'nid', 'time step': 'time'}), on='nid', how='left')
    node_label['label'] = node_label['label'].apply(lambda x: '3' if x == 'unknown' else x).astype(int) - 1
    node_label.head()

    merged_nodes_df = node_label.merge(
        df_features.rename(columns={'id': 'nid', 'time step': 'time'}).drop(columns=['time']), on='nid', how='left')
    merged_nodes_df.head()

    train_dataset = []
    test_dataset = []

    num_node_features = 0
    for i in range(49):
        nodes_df_tmp = merged_nodes_df[merged_nodes_df['time'] == i + 1].reset_index()
        nodes_df_tmp['index'] = nodes_df_tmp.index
        df_edge_tmp = df_edge_time_fin.join(
            nodes_df_tmp.rename(columns={'nid': 'source'})[['source', 'index']].set_index('source'), on='source',
            how='inner') \
            .join(nodes_df_tmp.rename(columns={'nid': 'target'})[['target', 'index']].set_index('target'), on='target',
                  how='inner', rsuffix='2') \
            .drop(columns=['source', 'target']) \
            .rename(columns={'index': 'source', 'index2': 'target'})
        x = torch.tensor(np.array(nodes_df_tmp.sort_values(by='index').drop(columns=['index', 'nid', 'label'])),
                         dtype=torch.float)
        edge_index = torch.tensor(np.array(df_edge_tmp[['source', 'target']]).T, dtype=torch.long)
        edge_index = to_undirected(edge_index)
        mask = nodes_df_tmp['label'] != 2
        y = torch.tensor(np.array(nodes_df_tmp['label']), dtype=torch.long)

        data = Data(x=x, edge_index=edge_index, mask=mask, y=y)
        num_node_features = data.num_node_features
        if i + 1 < 35:
            train_dataset.append(data)
        else:
            test_dataset.append(data)

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    return train_loader, test_loader, num_node_features

In [ ]:
import torch
from torch.nn import Parameter
from torch_geometric.nn import GCNConv
import torch.nn.functional as F


class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, use_skip=False):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels[0])
        self.conv2 = GCNConv(hidden_channels[0], 2)
        self.use_skip = use_skip
        if self.use_skip:
            self.weight = torch.nn.init.xavier_normal_(Parameter(torch.Tensor(num_node_features, 2)))

    def forward(self, data):
        x = self.conv1(data.x, data.edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, data.edge_index)
        if self.use_skip:
            x = F.softmax(x + torch.matmul(x, self.weight), dim=-1)
        else:
            x = F.softmax(x, dim=-1)
        return x

    def embed(self, data):
        x = self.conv1(data.x, data.edge_index)
        return x

In [ ]:
import torch
from torch.nn import LSTM
from torch_geometric.nn import GCNConv


class EvolveGCNO(torch.nn.Module):
    r"""An implementation of the Evolving Graph Convolutional without Hidden Layer.
    For details see this paper: `"EvolveGCN: Evolving Graph Convolutional 
    Networks for Dynamic Graph." <https://arxiv.org/abs/1902.10191>`_

    Args:
        in_channels (int): Number of filters.
        improved (bool, optional): If set to :obj:`True`, the layer computes
            :math:`\mathbf{\hat{A}}` as :math:`\mathbf{A} + 2\mathbf{I}`.
            (default: :obj:`False`)
        cached (bool, optional): If set to :obj:`True`, the layer will cache
            the computation of :math:`\mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
            \mathbf{\hat{D}}^{-1/2}` on first execution, and will use the
            cached version for further executions.
            This parameter should only be set to :obj:`True` in transductive
            learning scenarios. (default: :obj:`False`)
        normalize (bool, optional): Whether to add self-loops and apply
            symmetric normalization. (default: :obj:`True`)
        add_self_loops (bool, optional): If set to :obj:`False`, will not add
            self-loops to the input graph. (default: :obj:`True`)
    """
    def __init__(self, in_channels: int, improved: bool=False, cached: bool=False,
                 normalize: bool=True, add_self_loops: bool=True):
        super(EvolveGCNO, self).__init__()

        self.in_channels = in_channels
        self.improved = improved
        self.cached = cached
        self.normalize = normalize
        self.add_self_loops = add_self_loops
        self._create_layers()


    def _create_layers(self):

        self.recurrent_layer = LSTM(input_size = self.in_channels,
                                    hidden_size = self.in_channels,
                                    num_layers = 1)


        self.conv_layer = GCNConv(in_channels = self.in_channels,
                                  out_channels = self.in_channels,
                                  improved = self.improved,
                                  cached = self.cached,
                                  normalize = self.normalize,
                                  add_self_loops = self.add_self_loops,
                                  bias = False)

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor, 
                edge_weight: torch.FloatTensor=None) -> torch.FloatTensor:
        """
        Making a forward pass.

        Arg types:
            * **X** *(PyTorch Float Tensor)* - Node embedding.
            * **edge_index** *(PyTorch Long Tensor)* - Graph edge indices.
            * **edge_weight** *(PyTorch Float Tensor, optional)* - Edge weight vector.

        Return types:
            * **X** *(PyTorch Float Tensor)* - Output matrix for all nodes.
        """
        W = self.conv_layer.weight[None, :, :]
        W, _ = self.recurrent_layer(W)
        self.conv_layer.weight = torch.nn.Parameter(W.squeeze())
        X = self.conv_layer(X, edge_index, edge_weight)
        return X

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Parameter
#from torch_geometric_temporal.nn.recurrent import EvolveGCNO


class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, num_classes, dropout_rate=0.5):
        super(RecurrentGCN, self).__init__()
        self.dropout_rate = dropout_rate
        self.recurrent_1 = EvolveGCNO(node_features, num_classes)
        self.recurrent_2 = EvolveGCNO(node_features, num_classes)
        self.linear = torch.nn.Linear(node_features, num_classes)

    def forward(self, data):
        x = self.recurrent_1(data.x, data.edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.recurrent_2(x, data.edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.linear(x)
        return F.log_softmax(x, dim=-1)

    def embed(self, data):
        x = self.recurrent_1(data.x, data.edge_index)
        return x

In [ ]:
from collections import OrderedDict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter


class Tree(nn.Module):
    def __init__(self, depth, n_in_feature, used_feature_rate, n_class, jointly_training=True):
        super(Tree, self).__init__()
        self.depth = depth
        self.n_leaf = 2 ** depth
        self.n_class = n_class
        self.jointly_training = jointly_training

        # used features in this tree
        n_used_feature = int(n_in_feature * used_feature_rate)
        onehot = np.eye(n_in_feature)
        using_idx = np.random.choice(np.arange(n_in_feature), n_used_feature, replace=False)
        self.feature_mask = onehot[using_idx].T
        self.feature_mask = Parameter(torch.from_numpy(self.feature_mask).type(torch.FloatTensor), requires_grad=False)
        # leaf label distribution
        if jointly_training:
            self.pi = np.random.rand(self.n_leaf, n_class)
            self.pi = Parameter(torch.from_numpy(self.pi).type(torch.FloatTensor), requires_grad=True)
        else:
            self.pi = np.ones((self.n_leaf, n_class)) / n_class
            self.pi = Parameter(torch.from_numpy(self.pi).type(torch.FloatTensor), requires_grad=False)

        # decision
        self.decision = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(n_used_feature, self.n_leaf)),
            ('sigmoid', nn.Sigmoid()),
            ('relu', nn.ReLU()),
        ]))

    def forward(self, x):
        """
        :param x(Variable): [batch_size,n_features]
        :return: route probability (Variable): [batch_size,n_leaf]
        """
        if x.is_cuda and not self.feature_mask.is_cuda:
            self.feature_mask = self.feature_mask.cuda()

        feats = torch.mm(x, self.feature_mask)  # ->[batch_size,n_used_feature]
        decision = self.decision(feats)  # ->[batch_size,n_leaf]

        decision = torch.unsqueeze(decision, dim=2)
        decision_comp = 1 - decision
        decision = torch.cat((decision, decision_comp), dim=2)  # -> [batch_size,n_leaf,2]

        # compute route probability
        # note: we do not use decision[:,0]
        batch_size = x.size()[0]
        _mu = Variable(x.data.new(batch_size, 1, 1).fill_(1.))
        begin_idx = 1
        end_idx = 2
        for n_layer in range(0, self.depth):
            _mu = _mu.view(batch_size, -1, 1).repeat(1, 1, 2)
            _decision = decision[:, begin_idx:end_idx, :]  # -> [batch_size,2**n_layer,2]
            _mu = _mu * _decision  # -> [batch_size,2**n_layer,2]
            begin_idx = end_idx
            end_idx = begin_idx + 2 ** (n_layer + 1)

        mu = _mu.view(batch_size, self.n_leaf)

        return mu

    def get_pi(self):
        if self.jointly_training:
            return F.softmax(self.pi, dim=-1)
        else:
            return self.pi

    def cal_prob(self, mu, pi):
        """
        :param mu [batch_size,n_leaf]
        :param pi [n_leaf,n_class]
        :return: label probability [batch_size,n_class]
        """
        p = torch.mm(mu, pi)
        return p

    def update_pi(self, new_pi):
        self.pi.data = new_pi


class Forest(nn.Module):
    def __init__(self, n_tree, tree_depth, n_in_feature, tree_feature_rate, n_class, jointly_training=True):
        super(Forest, self).__init__()
        self.trees = nn.ModuleList()
        self.n_tree = n_tree
        for _ in range(n_tree):
            tree = Tree(tree_depth, n_in_feature, tree_feature_rate, n_class, jointly_training)
            self.trees.append(tree)

    def forward(self, x):
        probs = []
        for tree in self.trees:
            mu = tree(x)
            p = tree.cal_prob(mu, tree.get_pi())
            probs.append(p.unsqueeze(2))
        probs = torch.cat(probs, dim=2)
        prob = torch.sum(probs, dim=2) / self.n_tree

        return prob


class NeuralDecisionForest(nn.Module):
    def __init__(self, feature_layer, forest):
        super(NeuralDecisionForest, self).__init__()
        self.feature_layer = feature_layer
        self.forest = forest

    def forward(self, data):
        out = self.feature_layer(data)
        out = self.forest(out)
        return out

In [ ]:
import os
from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter


class TrainTest:

    def __init__(
            self,
            model,
            train_loader,
            val_loader,
            optimizer,
            loss_fn=nn.KLDivLoss(),
            temp=20.0,
            distil_weight=0.5,
            device="cpu",
            log=False,
            logdir="./Experiments",
    ):

        self.train_loader = train_loader
        self.val_loader = val_loader
        self.optimizer= optimizer
        self.temp = temp
        self.distil_weight = distil_weight
        self.log = log
        self.logdir = logdir

        if self.log:
            self.writer = SummaryWriter(logdir)

        try:
            torch.Tensor(0).to(device)
            self.device = device
        except:
            print(
                "Either an invalid device or CUDA is not available. Defaulting to CPU."
            )
            self.device = torch.device("cpu")

        try:
            self.model = model.to(self.device)
        except:
            pass
        try:
            self.loss_fn = loss_fn.to(self.device)
            self.ce_fn = nn.CrossEntropyLoss().to(self.device)
        except:
            self.loss_fn = loss_fn
            self.ce_fn = nn.CrossEntropyLoss()
            print("Warning: Loss Function can't be moved to device.")

    def train(
            self,
            epochs=10,
            plot_losses=True,
            save_model=True,
            save_model_pth="./models/dndf.pt",
    ):
        
        self.model.train()
        loss_arr = []
        illicit_f1_arr = []
        micro_avg_f1_arr = []
        illicit_precision_arr = []
        micro_avg_precision_arr = []
        illicit_recall_arr = []
        micro_avg_recall_arr = []
        length_of_dataset = len(self.train_loader.dataset)
        best_acc = 0.0
        self.best_model_weights = deepcopy(self.model.state_dict())

        save_dir = os.path.dirname(save_model_pth)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        print("Training... ")

        for ep in range(epochs):
            epoch_loss = 0.0
            correct = 0
            torch.manual_seed(ep)
            np.random.seed(42)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            for data in self.train_loader:

                data.x = data.x.to(self.device)
                label = data.y.to(self.device)
                mask = data.mask

                out = self.model(data)

                if isinstance(out, tuple):
                    out = out[0]

                pred = out.argmax(dim=1, keepdim=True)
                correct += pred.eq(label.view_as(pred)).sum().item()
                illicit_f1_arr.append(f1_score(pred[mask], label[mask], pos_label=1))
                micro_avg_f1_arr.append(f1_score(pred[mask], label[mask], average='micro'))
                illicit_precision_arr.append(precision_score(pred[mask], label[mask], pos_label=1))
                micro_avg_precision_arr.append(precision_score(pred[mask], label[mask], average='micro'))
                illicit_recall_arr.append(recall_score(pred[mask], label[mask], pos_label=1))
                micro_avg_recall_arr.append(recall_score(pred[mask], label[mask], average='micro'))

                loss = self.ce_fn(out[mask], label[mask])

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss

            epoch_acc = correct / length_of_dataset
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                self.best_model_weights = deepcopy(
                    self.model.state_dict()
                )

            if self.log:
                self.writer.add_scalar("Training loss", epoch_loss, epochs)
                self.writer.add_scalar("Training accuracy", epoch_acc, epochs)

            loss_arr.append(epoch_loss)
            print(
                'Epoch: {:1d}, Epoch Loss: {:.4f}, Illicit Precision: {:.4f}, Illicit Recall: '
                '{:.4f}, Illicit f1: {:.4f}, F1: {:.4f}, Precision: {:.4f}, Recall: {:.4f}' \
                    .format(ep + 1, epoch_loss, np.mean(illicit_precision_arr),
                            np.mean(illicit_recall_arr), np.mean(illicit_f1_arr), np.mean(micro_avg_f1_arr),
                            np.mean(micro_avg_precision_arr), np.mean(micro_avg_recall_arr)))

            self.post_epoch_call(ep)

        self.model.load_state_dict(self.best_model_weights)
        if save_model:
            torch.save(self.model.state_dict(), save_model_pth)
        if plot_losses:
            plt.plot(loss_arr)

    

    def _evaluate_model(self, model, verbose=False):
        """
        Evaluate the given model's accuaracy over val set.
        For internal use only.
        :param model (nn.Module): Model to be used for evaluation
        :param verbose (bool): Display Accuracy
        """
        model.eval()
        length_of_dataset = len(self.val_loader.dataset)
        correct = 0
        outputs = []
        illicit_f1_arr = []
        micro_avg_f1_arr = []
        illicit_precision_arr = []
        micro_avg_precision_arr = []
        illicit_recall_arr = []
        micro_avg_recall_arr = []

        seed_val = 35

        with torch.no_grad():
            for data in self.train_loader:

                torch.manual_seed(seed_val)
                np.random.seed(seed_val)
                torch.backends.cudnn.deterministic = True
                torch.backends.cudnn.benchmark = False

                data.x = data.x.to(self.device)
                target = data.y.to(self.device)
                mask = data.mask

                output = model(data)

                if isinstance(output, tuple):
                    output = output[0]
                outputs.append(output)

                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
                accuracy = correct / length_of_dataset
                illicit_f1_arr.append(f1_score(pred[mask], target[mask], pos_label=1))
                micro_avg_f1_arr.append(f1_score(pred[mask], target[mask], average='micro'))
                illicit_precision_arr.append(precision_score(pred[mask], target[mask], pos_label=1))
                micro_avg_precision_arr.append(precision_score(pred[mask], target[mask], average='micro'))
                illicit_recall_arr.append(recall_score(pred[mask], target[mask], pos_label=1))
                micro_avg_recall_arr.append(recall_score(pred[mask], target[mask], average='micro'))

                if verbose:
                    print("-" * 80)
                    print(f"Iteration: {seed_val-34}")
                    print("-" * 80)
                    print("Illicit F1: {:.4f}".format(f1_score(pred[mask], target[mask], pos_label=1)))
                    print("Illicit Precision: {:.4f}".format(precision_score(pred[mask], target[mask], pos_label=1)))
                    print("Illicit Recall: {:.4f}".format(recall_score(pred[mask], target[mask], pos_label=1)))
                    print("Micro Avg F1: {:.4f}".format(f1_score(pred[mask], target[mask], average='micro')))
                    print("Micro Avg Precision: {:.4f}".format(precision_score(pred[mask], target[mask], average='micro')))
                    print("Micro Avg Recall: {:.4f}".format(recall_score(pred[mask], target[mask], average='micro')))

                seed_val += 1

        print("-" * 80)
        print("-" * 80)
        print("Final Result")
        print("-" * 80)
        print("-" * 80)
        
        print("Illicit F1: {:.4f}".format(np.mean(illicit_f1_arr)))
        print("Illicit Precision: {:.4f}".format(np.mean(illicit_precision_arr)))
        print("Illicit Recall: {:.4f}".format(np.mean(illicit_recall_arr)))
        print("Micro Avg F1: {:.4f}".format(np.mean(micro_avg_f1_arr)))
        print("Micro Avg Precision: {:.4f}".format(np.mean(micro_avg_precision_arr)))
        print("Micro Avg Recall: {:.4f}".format(np.mean(micro_avg_recall_arr)))
        return outputs, accuracy

    def evaluate(self):
        """
        Evaluate method for printing accuracies of the trained network
    
        """
        model = deepcopy(self.model).to(self.device)
        _, accuracy = self._evaluate_model(model=model, verbose=False)
        
        return accuracy


    def post_epoch_call(self, epoch):
        """
        Any changes to be made after an epoch is completed.
        :param epoch (int) : current epoch number
        :return            : nothing (void)
        """

        pass

In [ ]:
import time
import tracemalloc


def get_memory_and_execution_time_details(func):
    tracemalloc.start()
    start_time = time.time()
    func()
    exec_time = time.time() - start_time
    print("Model Evaluation Time: ")
    print(exec_time)
    current, peak = tracemalloc.get_traced_memory()
    print(f"Current memory usage is {current / 10 ** 3}KB; Peak was {peak / 10 ** 3}KB")
    tracemalloc.stop()

    return current, peak, exec_time

In [ ]:
train_loader, test_loader, num_node_features = train_test_split()
lr = 10e-5
weight_decay = 5e-4
epochs = 10

In [ ]:
import time
import torch
import torch.optim as optim
# Deep Neural Decision Forest
feat_layer = RecurrentGCN(node_features=num_node_features, num_classes=2, dropout_rate=0.65)
forest = Forest(n_tree=80, tree_depth=8, n_class=2, n_in_feature=2, tree_feature_rate=0.65)
ndf = NeuralDecisionForest(feat_layer, forest)
optimizer_ndf = optim.Adam(ndf.parameters(), lr=lr, weight_decay=weight_decay, amsgrad=True)


In [ ]:
dndf = TrainTest(ndf, train_loader, test_loader, optimizer_ndf)
dndf.train(epochs=epochs, plot_losses=True, save_model=True,
                             save_model_pth='./models/dndf.pt')  

In [ ]:
ndf.load_state_dict(torch.load("./models/dndf.pt"))
get_memory_and_execution_time_details(dndf.evaluate)  